In [1]:
words = open('../names.txt','r').read().splitlines()

In [2]:
import torch

In [28]:
# stoi and itos dicts
chars = ['.']+sorted([chr(char) for char in range(ord('a'),ord('z')+1)])+['<s>']
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}

In [19]:
N = torch.zeros(28,28,27) # cuz I wouldn't expect <s> for the 3rd dimension. and anyway its just a filler character.

In [20]:
#filling the N array.
for word in words:
    chs = ['.']+['<s>']+list(word)+['.']
    for cha,chb,chc in zip(chs,chs[1:],chs[2:]):
        # print(f'cha,chb,chc : {cha,chb,chc}')
        i,j,k = stoi[cha],stoi[chb],stoi[chc]
        N[i,j,k]+=1

In [109]:
N.sum() # the count will be same as you can imagine each increment is like sliding over the name and turns out with your current approach you are sliding as much as when you were doing bigrams with just window size of 3.


tensor(228146.)

In [110]:
P = (N+1).float()
# print(P.sum(dim=2).shape) #wrong # torch.Size([28, 28])
# print(P.shape) # torch.Size([28, 28, 27])
# P.sum(dim=2,keepdim=True).shape #torch.Size([28, 28, 1])
P/=P.sum(dim=2,keepdim=True)

In [111]:
log_likelihood=0.0
ncount=0
for w in words:
    # chs = ['.']+['<s>']+list(word)+['.']
    chs = ['.']+['<s>']+list(w)+['.'] #bug identified.
    for cha,chb,chc in zip(chs,chs[1:],chs[2:]):
        i,j,k = stoi[cha],stoi[chb],stoi[chc]
        prob = P[i,j,k]
        log_likelihood+=torch.log(prob)
        ncount+=1
neg_log_likelihood = -log_likelihood
mean_neg_log_likelihood = neg_log_likelihood/ncount

In [112]:
print(f'{neg_log_likelihood=}')
print(f'{mean_neg_log_likelihood=}')
print(ncount)

neg_log_likelihood=tensor(504653.)
mean_neg_log_likelihood=tensor(2.2120)
228146


In [114]:
g = torch.Generator().manual_seed(2147483647)


for _ in range(10):
    nameList = []
    ix,iy =0,27
    while True:
        p = P[ix,iy]
        iz = torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
        if iz==0:
            break
        nameList.append(itos[iz])
        ix,iy=iy,iz
    print(''.join(nameList))

junide
jakasid
prelay
adin
kairritoper
sathen
sameia
yanileniassibduinrwin
lessiyanayla
te


Implementing the neural net.

In [116]:
nnNames = list(words)

In [117]:
import random
random.shuffle(nnNames)

train_size = int(0.8 * len(nnNames))
dev_size = int(0.1 * len(nnNames))
test_size = len(nnNames) - train_size - dev_size

train_data = nnNames[:train_size]
dev_data = nnNames[train_size:train_size + dev_size]
test_data = nnNames[train_size + dev_size:]

In [162]:
xs,ys,zs = [],[],[]

# for name in nnNames: # should be using train data here
for name in train_data:
    # n = ['.']+['<s>']+list(word)+['.'] #same bug using word instead of name...
    n = ['.']+['<s>']+list(name)+['.']
    for cha,chb,chc in zip(n,n[1:],n[2:]):
        xs.append(stoi[cha])
        ys.append(stoi[chb])
        zs.append(stoi[chc])

xs,ys,zs = torch.tensor(xs),torch.tensor(ys),torch.tensor(zs)

In [163]:
xs.shape,ys.shape,zs.shape

(torch.Size([182636]), torch.Size([182636]), torch.Size([182636]))

In [120]:
import torch.nn.functional as F

In [121]:
# was thinking like how we used W as the same dimension as N in bigram I think I should do the same here
W = torch.randn((28,28,27),requires_grad=True)


In [122]:
cpyW = W.clone().detach()

Backpropagation

In [123]:
zenc = F.one_hot(zs,num_classes=27).float()
zenc.shape

torch.Size([182636, 27])

In [124]:
W[1,1,:]

tensor([-1.6171, -0.6645,  0.5212,  0.0623,  0.4992, -0.6868,  0.9780,  0.1445,
        -0.6494,  0.5022,  0.0471,  0.8851,  0.3390, -0.8820, -0.7232, -1.3166,
        -0.1695, -0.1290,  2.2472, -0.1777,  1.5775, -1.1554, -1.1667,  0.0774,
        -1.1502,  1.0874, -0.8565], grad_fn=<SliceBackward0>)

In [125]:
W[[1,2],[1,2]]

tensor([[-1.6171, -0.6645,  0.5212,  0.0623,  0.4992, -0.6868,  0.9780,  0.1445,
         -0.6494,  0.5022,  0.0471,  0.8851,  0.3390, -0.8820, -0.7232, -1.3166,
         -0.1695, -0.1290,  2.2472, -0.1777,  1.5775, -1.1554, -1.1667,  0.0774,
         -1.1502,  1.0874, -0.8565],
        [ 0.4748,  1.2822,  0.3095, -0.7268,  0.8141,  1.4176, -0.5450, -0.0938,
          0.1917,  1.4946, -2.2174,  1.6186,  0.4499, -1.6243, -0.6266, -0.2850,
          2.4141, -1.2275,  1.9610, -1.0391,  0.5133,  0.3828, -1.1896, -0.9299,
         -0.4605, -0.1121,  0.1388]], grad_fn=<IndexBackward0>)

In [310]:
step_size = 0.01*1000
for i in range(10):
    #forward pass
    logits  =W[xs,ys,:]
    loss = F.cross_entropy(logits,zenc)

    #backward pass
    W.grad = None
    loss.backward()

    #update 
    W.data+=W.grad*step_size*-1

    print(loss)

tensor(2.3181, grad_fn=<DivBackward1>)
tensor(2.3181, grad_fn=<DivBackward1>)
tensor(2.3180, grad_fn=<DivBackward1>)
tensor(2.3179, grad_fn=<DivBackward1>)
tensor(2.3179, grad_fn=<DivBackward1>)
tensor(2.3178, grad_fn=<DivBackward1>)
tensor(2.3177, grad_fn=<DivBackward1>)
tensor(2.3176, grad_fn=<DivBackward1>)
tensor(2.3176, grad_fn=<DivBackward1>)
tensor(2.3175, grad_fn=<DivBackward1>)


In [311]:
#sampling
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out = []
    ix,iy= 0,27 # ix = . iy = <s>
    while True:
        l_logits = W[ix,iy] # gives the 3rd dimension of the weight corresponding to each 27 neurons of the pair ix,iy
        l_counts = l_logits.exp()
        l_p = l_counts/l_counts.sum()

        iz = torch.multinomial(l_p,num_samples=1,replacement=True,generator=g).item()
        if iz==0:
            break
        out.append(itos[iz])
        ix,iy = iy,iz
    print(''.join(out))

junjdmdianasid
prosay
adin
kai
ritonian
juwa
kalinaaurinileniassdbduinrwibwxdskjyinayla
te
farmumtsjfortumj
poyashiah


Okay so its a little better now.

Turns out ther was a bug in the loop that prepared the xs,ys and zs.

Testing of the model on the test set.

In [321]:
tx,ty,tz = [],[],[]
for name in test_data:
    n = ['.']+['<s>']+list(name)+['.']
    for cha,chb,chc in zip(n,n[1:],n[2:]):
        tx.append(stoi[cha])
        ty.append(stoi[chb])
        tz.append(stoi[chc])

tx,ty,tz = torch.tensor(tx),torch.tensor(ty),torch.tensor(tz)
tzenc = F.one_hot(tz,num_classes=27).float()

In [322]:
tx.shape,ty.shape,tz.shape,tzenc.shape

(torch.Size([22779]),
 torch.Size([22779]),
 torch.Size([22779]),
 torch.Size([22779, 27]))

Evaluating loss

In [323]:
t_logits = W[tx,ty]
loss = F.cross_entropy(t_logits,tzenc)
loss

tensor(2.3320, grad_fn=<DivBackward1>)